In [12]:
using HomotopyContinuation, HomologyInferenceWithWeakFeatureSize, DynamicPolynomials
hwfs = HomologyInferenceWithWeakFeatureSize
data_path = escape_string("paper_data/")
threshold = 1e-7

# Order of bottlenecks, change for other orders
k = 3

filename_root = "bots"*string(k)*"_"

# Uncomment if you have re-run the corresponding Bertini script
# data_path = escape_string("paper_data/bertini_inputs/"*str(k)*"_bottlenecks/")
# filename_root = ""

"bots3_"

In [13]:
num_vars = 3
@polyvar q[1:num_vars]
x = q[1]
y = q[2]
z = q[3]
F = [-3 + 3*z^4 + 4*y + 8*y^2 - 6*y^3 + 7*y^4 + 8*x - 5*x*y - 8*x*y^2 - 4*x^2 + 2*x^2*y - 8*x^3 + 4*x^4]
singular_points = hwfs.parse_bertini_file(data_path*filename_root*"singular_solutions")
singular_points = [hwfs.quartic_bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in singular_points]
nonsingular_points = hwfs.parse_bertini_file(data_path*filename_root*"nonsingular_solutions")
nonsingular_points = [hwfs.quartic_bertini_convert_solution_point_to_standard_form(point,num_vars,k) for point in nonsingular_points]

filtered_nonsingular_points = [point for point in nonsingular_points if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]
# This filters out some complex, isolated, and singular contributors
function check_if_solution_has_third_coordinate_zero(point)
    if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)
        return false
    end
    for endpoint in point["endpoints"]
        if real(abs(endpoint[3])) <= threshold
            return true
        end
    end
    return false
end
filtered_singular_points = [point for point in singular_points if !check_if_solution_has_third_coordinate_zero(point)]


all_points = [filtered_singular_points;filtered_nonsingular_points]
nondegenerate_points = [point for point in all_points if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)]
real_positive_D_value_points = [point for point in nondegenerate_points if hwfs.real_and_positive_d_value_check(point;threshold=threshold)]
can_be_distance_filtered = [point for point in real_positive_D_value_points if hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]
cannot_be_distance_filtered = [point for point in real_positive_D_value_points if !hwfs.check_if_solution_has_real_endpoints(point;threshold=threshold)]

distance_system_for_filtering,parameters_for_filtering = hwfs.minimum_distance_start_system(F,q)
start = randn(ComplexF64,num_vars+length(F))
initial_system = subs(distance_system_for_filtering,parameters_for_filtering => start)
initial_solution = solutions(solve(initial_system,start_system = :polyhedral,parameters=:conservative))
distances_isolated = [Inf]
if length(can_be_distance_filtered) > 0
    distances_isolated = [hwfs.filter_solution_to_bottleneck(solution,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold) for solution in can_be_distance_filtered]
    distances_isolated = [dist for dist in distances_isolated if dist!=false]
    if length(distances_isolated) == 0
        distances_isolated = [Inf]
    end
end
distances_not_isolated = [Inf]
if length(cannot_be_distance_filtered) > 0 
    distances_not_isolated = [real(sqrt(hwfs.apply_distance_squared_to_solution(point))) for point in cannot_be_distance_filtered]
end




In [14]:
# This number is the total number of solutions computed on B_k(F)
length(singular_points)+length(nonsingular_points)

40672

In [15]:
# This counts the solutions which are contained in \Gamma_k(F)
length(singular_points)+length(nonsingular_points)-length([point for point in [singular_points;nonsingular_points] if !hwfs.check_if_solution_is_degenerate(point;threshold=threshold)])

8191

In [16]:
# The number of non-isolated solutions contributing 
# to the weak feature size
length(cannot_be_distance_filtered)

0

In [23]:
# Real geometric bottlenecks: There is 1 geometric bottleneck of multiplicity 
# 3 when k = 2
println([real(point["circumcenter"]) for point in can_be_distance_filtered if hwfs.filter_solution_to_bottleneck(point,length(q),k,distance_system_for_filtering,start,parameters_for_filtering,initial_solution;threshold=threshold)!=false])

[[1.3196191723703588, 0.027546060015504192, -3.4109849049807754e-15], [1.3196191723703943, 0.02754606001550222, -3.032214242335742e-15], [1.3196191723703812, 0.02754606001549534, -1.960025016786294e-15], [-0.43908836090446646, -0.10204060426368403, -1.876643012704299e-15], [-0.439088360904469, -0.10204060426369221, -2.4676735259138988e-15]]


In [24]:
# Critical values from isolated geometric k-bottlenecks
minimum(distances_isolated)

0.6352420284771654